#STEP 1 BW_assignment

In [ ]:
from Bio import pairwise2
from Bio.pairwise2 import *
#import Biopython for alignment

def readin_files(file):
    data_list = []
    with open(file) as open_file:
        for line in open_file:
            data_list.append(line[:-1])
    return data_list
#defines the reading of files

assignment = readin_files('assignment.list')
#read in all the assignments

#######################
#Define the fasta file#
#######################
target = 'target.fasta'

for line in open(target, 'r').readlines():
    if line[0] == ">":
        target_name = line[1:-1]
    else:
        target_fasta = line[:-1]
print(target_name)
print(target_fasta)

In [ ]:
print('Starting assignment for', target_name)
for assign in assignment:
    assign = assign.split('\t')
    if target_name == assign[0]:
        bw_assign = "> " + assign[2]
    if target_name == assign[2]:
        bw_assign = "> " + assign[2]
print(target_name, " was assign as ", bw_assign)
#Here the script tries to find the model in the assignment list, so to define bw_assign for the later BW file

gpcrdb = open('all_gpcrdb_num.data', 'r')
gpcrdb_lines = gpcrdb.readlines()
for i in range (0, len(gpcrdb_lines)):
    if gpcrdb_lines[i][:-1] == bw_assign:
        position = i
print(bw_assign, ' can be found in line ',position)
#The script gets the information, where in the BW list to find the needed residues

fasta_bw = ''
fasta = gpcrdb_lines[position + 1]
fasta = fasta.split('\t')
fasta[-1] = fasta[-1][:-1]
bw = gpcrdb_lines[position + 2]
bw = bw.split('\t')
bw[-1] = bw[-1][:-1]
for ele in fasta:
    fasta_bw = fasta_bw + ele[0]
#From the BW all data list the sequence for BW numbering is taken, defined and a list with the names provided

alignments = pairwise2.align.globalxx(target_fasta, fasta_bw)
#use of Biophython lib and the module pairwise 2
aligned_bw = alignments[0][1]
aligned_pdb = alignments[0][0]
#the alignment will put '-' in spaces were there are differences

offset_bw = 0
offset_pdb = 0
if aligned_pdb[0] == '-':
    for residue in aligned_pdb:
        if residue != '-':
            break
        else:
            offset_pdb += 1
else:
    for residue in aligned_bw:
        if residue != '-':
            break
        else:
            offset_bw += 1
#need to define an offset value (bw sequence starts earlier or later)

bw_for_alignment = []           
#print(offset_bw, offset_pdb)


offset_aligned_in_fasta = 0
offset_bw_in_fasta = 0

f = open(f"{bw_assign[2:]}.bw", 'w')

bw_numbering_final = []
if offset_bw > offset_pdb: 
    for i in range(offset_bw, len(target_fasta)):
        if aligned_bw[i] == '-':
            offset_aligned_in_fasta += 1 
        else:
            try:
                if aligned_pdb[i] == '-':
                    offset_bw_in_fasta += 1 
                else:
                    print(aligned_pdb[i], i + 1 - offset_bw_in_fasta, bw[i - offset_bw - offset_aligned_in_fasta], file=f)
                    bw_numbering_final.append((aligned_pdb[i], i + 1 - offset_bw_in_fasta, bw[i - offset_bw - offset_aligned_in_fasta]))
            except:
                pass
#Here happens the magic and it is complicated
#We need two further offset values, one k for the bw numbering and z for when in the crystal structure we have
#another residue (complicated, but the assignment will make from ex. VASQAS BW and VASNAS Crystal -> VAS-QAS and VASN-AS)
#The code will correct this with z (DON'T ASK ME, IT'S MAGIC)

else:
    for i in range(offset_pdb, len(target_fasta)):
        if aligned_bw[i] == '-':
            k  += 1 
        else:
            try:                    
                if aligned_pdb[i] == '-':
                    z += 1 
                else:
                    print(aligned_pdb[i], (i + 1 - offset_pdb - offset_bw_in_fasta), bw[i - offset_aligned_in_fasta], file=f)
                    bw_numbering_final.append((aligned_pdb[i], (i + 1 - offset_pdb - offset_bw_in_fasta), bw[i - offset_aligned_in_fasta]))
            except:
                pass
#prints AA, number in Rosetta and BW Numbering
f.close()       

#STEP 2 Make_cst_files

In [ ]:
import random

def random_para():
    global weight, distance, harmonic, constant
    weight = random.randint(2,4)
    distance = (1+float(random.randint(3,7)/100))
    harmonic = (random.randint(4,8)/10)
    constant = str(-(random.randint(1,10)))
random_para()
#initialize the random function and generate a first parameter set

In [ ]:
bw_file = 'ADRB1.bw'
distance_pair = []
for line in open('selected_pairs.list', 'r').readlines():
    line = line.split()
    distance_pair.append(line)
for pair in distance_pair:
    print(pair)
#opens the selected distances with res low, res high and distance in A

number_of_constraint_files = 10
#########################################
#    Define the number of constraints   #
#This is similar to the number of models#
#########################################

In [ ]:
for i in range(1, (number_of_constraint_files+1)):
    cst_file_name = bw_file[:-3] + '_' + str(i) + '.cst'
    cst_file = open(cst_file_name, 'w')
    #build a new cst file 
    constraint = []
    tetrahedron_counter = 0
    for pair in distance_pair:
        tetrahedron_counter += 1
        file = open(bw_file, 'r')
        res_num = []
        for line in file:
            line = line.split()
            if pair[0] == line[2]:
                res_num.append(line[1])
            if pair[2] == line[2]:
                res_num.append(line[1])
        #get the information for the bw pair and corresponding number in the fasta --> final Rosetta res number
        try:
            print('AtomPair \t CA \t', res_num[0], '\t CA \t ', res_num[1], '\t SCALARWEIGHTEDFUNC \t', 
            weight, '\t SUMFUNC \t 2 \t HARMONIC \t', str(round(float(pair[3])*distance, 4)), 
            '\t', harmonic, '\t CONSTANTFUNC \t', constant, file=cst_file)

        except:
            print('#There was an error with', res_num[0], res_num[1])
            print('#There was an error with', res_num[0], res_num[1])
        #write the constraint line into a file, if there is an error --> missing bw number, catch the except 
        if tetrahedron_counter == 6:
            print('#The selected parameters were:', weight, distance, harmonic, constant, file=cst_file)
            tetrahedron_counter = 0 
            random_para()
        #after one tetrahedron (6 pairs of residues), write the selected parameter and generate new ones
    cst_file.close()
    
#simple script, opens new file, starts for each distance pair a new for loop --> opens bw file, searchs for fitting number

In [ ]:
#Bonus: Write a bash script to copy the constraints and rename XML script
bash_script = open('copy_cst_and_xml.sh', 'w')
print('cp *.cst ../Step3_RosettaGPCR/input/', file=bash_script)
for i in range(1, (number_of_constraint_files+1)):
    cst_file_name = bw_file[:-3] + '_' + str(i) + '.cst'
    xml_file_name = 'rosetta_cm' + str(i) + '.xml'
    print(f'sed "s/constraint_number/{cst_file_name}/g" ../Step3_RosettaGPCR/input/rosetta_cm.xml >>../Step3_RosettaGPCR/input/{xml_file_name}',
         file=bash_script)
bash_script.close()

#STEP 4 Calculate_volume_and_filtering

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits import mplot3d
from scipy.spatial import *
import pandas as pd

import os
os.system('ls *.pdb > pdb.list')
#generate a list with all pdbs

def readin_files(file):
    data_list = []
    with open(file) as open_file:
        for line in open_file:
            data_list.append(line[:-1])
    return data_list
#defines the reading of files

In [ ]:
model = readin_files('pdb.list')
volume_residues = readin_files('selected_res_vol.list')
bw_numbering = readin_files('ADRB1.bw')
#read in all the necessary files and assignments

relevant_vol_res = []
for residue in bw_numbering:
    res = residue.split()
    if res[2] in volume_residues:
        relevant_vol_res.append(res[1])
print(relevant_vol_res)
#build a Rosetta number list

In [ ]:
for structure in model:
    xyz_name = f"{structure}.xyz"
    xyz_file = open(xyz_name, 'w')
    for lines in open(structure, 'r').readlines():
        line = lines.split()
        try:
            if line[5] in relevant_vol_res and line[4] == 'A' and line[2] == 'CA': 
                print(*line[3:9], sep='\t', file=xyz_file)
        except:
            pass
    xyz_file.close()
#selects from all pdbs the relevant residues (CA atom) and the x, y and z coordinates

In [ ]:
xyz = []
for structure in model:
    xyz.append((structure + ".xyz"))

counter = 0
for ele in xyz:
    vol_file = open(f"{ele}.vol", "w")
    points = []
    file = open(ele, 'r')
    try:
        for line in file:
            line = line.split()
            if "NA" not in line:
                point = line[-3:]
            points.append(point)
        hull = ConvexHull(points)

        print(ele, hull.volume, file=vol_file)
        vol_file.close()
        counter += 1
        print(counter)
    except:
        print(ele, "NA", file=f)
#For calculating the volume of the 10 res and saving them in a new file
os.system('find . -name "*vol" -exec cat {} > all_vol.list \;')
#One final file for all volumes
os.system('find . -name "*.pdb" | xargs grep -E "pose" > score.list \;')

In [ ]:
df_volume = pd.read_csv('all_vol.list', names=['description', 'volume'], sep='\s+')
df_volume['description'] = df_volume['description'].map(lambda x: str(x)[:-4])
df_score = pd.read_csv('score.list', header=None, sep='\s+')
df_score['description'] = df_score[0].map(lambda x: str(x)[2:-5])
df_score_final = df_score[[23, 'description']]
merged = pd.merge(left=df_volume, right=df_score_final, on='description')
merged = merged.rename(columns={23: 'score'})
merged
#read in the volume and the score files and generates a merged dataframe

In [ ]:
merged['volume'] = merged['volume'].astype(float)
merged['score'] = merged['score'].astype(float)
mean_vol = merged['volume'].mean()

print(mean_vol)
selected = merged.loc[(merged['volume'] > (mean_vol-200)) & (merged['volume'] < (mean_vol+200))]
selected.sort_values(by=['score'], inplace=True, ascending=True)
selected.to_csv('within_200.list', sep='\t', index=None)
os.system(f'echo "\nThe mean_volume was {round(mean_vol, 2)}" >> within_200.list')
best5 = selected.head(5)
best5.to_csv('best5.list', sep='\t', index=None)
os.system(f'echo "\nThe mean_volume was {round(mean_vol, 2)}" >> best5.list')